# Obtención de datos y conección a API

In [10]:
import requests
import pandas as pd

## Búsqueda de indicadores

In [11]:
url='http://api.worldbank.org/v2/es/indicator'
def consultar_indicadores(pagina=1):
    argumentos= {
        'page':pagina,
        'per_page':10000,
        'format':'json'
    }
    return requests.get(url,params=argumentos).json()

indicadores = consultar_indicadores()

# la primera parte de la respuesta 
# contiene información sobre las páginas
paginas = indicadores[0]["pages"]

# Reasigno la variable únicamente a los datos de mi interés 
# es una lista de diccionarios
indicadores = indicadores[1]

# recorro todás las páginas que tiene la respuesta 
# y lo voy anexando a la lista principal
for pagina in range(2, paginas+1):
    indicadores.extend(consultar_indicadores(pagina)[1])

In [12]:
indicators = pd.json_normalize(indicadores)
# Dev data para trabajo offline
# indicators.to_csv('../datasets/indicadores.csv')

In [13]:
interes = input("Indicador de interés: ")
indicators.loc[indicators.name.str.contains(interes,case=False),
     ['id','name','sourceNote']].to_csv('temp.csv')

## Selección y extracción de un solo indicador

Selección de un solo país bajo estudio. Si se eligen todos los países la API tarda un poco más de 30 segundos en responder una vez terminado el ciclo for. Y esa carga inicial podría molestar al usuario final.

In [14]:
datos=pd.read_csv('../datasets/esperanza_de_vida.csv')
paises = datos[['countryiso3code','country.value']].groupby('countryiso3code').first()
paises.dropna(inplace=True) #La búsqueda no funciona sin esto XD

seleccion = input("Ingrese un nombre de país: ")
try:
    pais = paises.loc[paises['country.value'].str.contains(seleccion, case=False)].index[0]
except:
    print("No se encontró el país, intente de nuevo")

In [15]:
# Establecemos los parámetros que queremos acceder.
indicador = 'SP.DYN.LE00.IN'
#pais = 'all' # Está comentado porque lo recibe del bloque anterior
iyear = '1980'
fyear = '2022'

# Página de la api y path al recurso solicitado
api_url = ' http://api.worldbank.org/v2/es'
path = f'/country/{pais}/indicator/{indicador}'

url = api_url + path

# Creamos el diccionario con los parametros 
# para el método get
def consultar(pagina = 1):
    args= {
        "date":iyear+":"+fyear,
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
    return requests.get(url,params=args).json()

dict_data = consultar()

In [16]:
# La primera parte de la respuesta nos indica en 
# cuantas páginas de encuentra la información
paginas=dict_data[0]["pages"]
paginas # Retornamos el número de páginas

1

In [17]:
# La segunda parte nos retorna una lista de diccionarios 
# con la información que queríamos
datos=dict_data[1]
datos

[{'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2021',
  'value': None,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2020',
  'value': 76.813,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2019',
  'value': 76.667,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.DYN.LE00.IN',
   'value': 'Esperanza de vida al nacer, total (años)'},
  'country': {'id': 'AR', 'value': 'Argentina'},
  'countryiso3code': 'ARG',
  'date': '2018',
  'value': 76.52,
  'uni

In [18]:
# Agregamos los valores de las otras páginas a 
# nuestra lista de diccionarios
for pagina in range(2,paginas+1):
    datos.extend(consultar(pagina)[1])

In [19]:
# Verifico que lo obtenido sea una lista de diccionarios
print(type(datos))
print(type(datos[1]))

<class 'list'>
<class 'dict'>


In [20]:
# Creo el DataFrame con todos los datos
data = pd.json_normalize(datos)

In [21]:
data[['date','value','country.value']]

,date,value,country.value
0,2021,NaN,Argentina
1,2020,76.813,Argentina
2,2019,76.667,Argentina
3,2018,76.520,Argentina
4,2017,76.372,Argentina
5,2016,76.221,Argentina
6,2015,76.068,Argentina
7,2014,75.913,Argentina
8,2013,75.756,Argentina
9,2012,75.598,Argentina


In [22]:
# Creo un dataset para trabajar offline
#data.to_csv('../datasets/esperanza_de_vida.csv')

# EDA

Verifico los datos faltantes en la respuesta de la API en español. Son dós nombres de países o grupo de países, con el código AFW y AFE, para Africa ocidental y central y Africa del este y del sur, respectivamente. En resumen: Regiones de Africa. No sé si lo vamos a incluir

In [23]:
data[data.countryiso3code.str.contains('AFW')]

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
